In [1]:
import torch
from datasets import load_dataset
from datasets import load_from_disk

# from d2l import torch as d2l


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# 定义数据集
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, split):
        # dataset = load_dataset('lansinuote/ChnSentiCorp', keep_in_memory=True)
        dataset = load_from_disk('./data/ChnSentiCorp')
        
        def f(data):
            return len(data['text']) > 30
        
        self.dataset = dataset.filter(f)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset[i]['text']
        return text

In [3]:
dataset = Dataset('train')

len(dataset) # , dataset[0]

Loading cached processed dataset at /home/mylady/code/python/DL-pytorch/apps/huggingface/data/ChnSentiCorp/validation/cache-cf45964edee402a8.arrow
Loading cached processed dataset at /home/mylady/code/python/DL-pytorch/apps/huggingface/data/ChnSentiCorp/test/cache-c6f7400aef16ddba.arrow
Loading cached processed dataset at /home/mylady/code/python/DL-pytorch/apps/huggingface/data/ChnSentiCorp/train/cache-478819d08c52879a.arrow


3

In [8]:
from transformers import BertTokenizer
from transformers import BertModel


# 加载字典和分词工具
token = BertTokenizer.from_pretrained('bert-base-chinese')

print(token)


# 加载预训练模型
pretrained = BertModel.from_pretrained('bert-base-chinese').to(device)

BertTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
def collate_fn(data):
    
    # print(data)
    # print('data长度: ', len(data))
    # 编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=data,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=30,
                                   return_tensors='pt',
                                   return_length=True)

    # print('编码后的data打印: ', data.keys())
    
    # input_ids:编码之后的数字
    # attention_mask:是补零的位置是0,其他位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']

    # 把第15个词固定替换为mask
    # print('第15个词: %s' % input_ids[:, 15])
    
    # 这里直接使用了编码后的数据作为真实预测值
    labels = input_ids[:, 15].reshape(-1).clone()
    input_ids[:, 15] = token.get_vocab()[token.mask_token]

    # print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, labels

## 数据加载器

In [15]:
# 数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset['train'],
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True
                                    )


for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
    print('当前loader数据量: ', len(loader))
    print('解码input_ids: ', token.decode(input_ids[0]))
    print('labels: ', token.decode(labels[0]))
    print('参数打印: ', input_ids.shape, attention_mask.shape, token_type_ids.shape, labels.shape)
    print("")
    if i >= 1:
        break

当前loader数据量:  574
解码input_ids:  [CLS] 这 本 书 有 它 的 局 限 ， 因 为 国 情 不 [MASK] 背 景 各 异 ， 而 且 作 者 也 不 能 完 [SEP]
labels:  同
参数打印:  torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16])

当前loader数据量:  574
解码input_ids:  [CLS] 帮 朋 友 买 的 ， 和 我 的 [UNK] 放 一 起 ， [MASK] 别 不 大 ， 稍 微 厚 点 ， 远 看 几 乎 [SEP]
labels:  区
参数打印:  torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16, 30]) torch.Size([16])



## 定义下游任务模型

In [20]:
# 定义下游任务模型
class Model(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.decoder = torch.nn.Linear(768, token.vocab_size, bias=False)
        self.bias = torch.nn.Parameter(torch.zeros(token.vocab_size))
        self.decoder.bias = self.bias

    def forward(self, input_ids, attention_mask, token_type_ids):
        
        # pretrained = pretrained.to(device)

        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                             attention_mask=attention_mask,
                             token_type_ids=token_type_ids)
            pass
        
        out = self.decoder(out.last_hidden_state[:, 15])
        return out


# 加载保存的模型
model_save_path = 'chinese_full_vacant_mission_2023_4_10.pt'
model = Model()
model.load_state_dict(torch.load(model_save_path))

# 模型转移到GPU上
model.to(device)

# list(model.parameters())[0].device  # device(type='cuda', index=0)

device(type='cuda', index=0)

## 模型测试

In [18]:
#测试
def test_calculate():
    
    model.eval()
    correct = 0
    total = 0

    loader_test = torch.utils.data.DataLoader(dataset=dataset['test'],
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader_test):
        
        if i == 15:
            break

        with torch.no_grad():
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            labels = labels.to(device)
            
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)
            pass
        
        out = out.argmax(dim=1).cpu()
        labels = labels.cpu()
        
        correct += (out == labels).sum().item()
        total += len(labels)

        print('序: %s 输入内容: %s' % (i, token.decode(input_ids[0])))
        print('解码: ', token.decode(labels[0]))
        
    print('acc: %.2f' % (correct / total))
    pass

In [19]:
# 预测
test_calculate()

序: 0 输入内容: [CLS] 该 酒 店 实 际 是 兰 州 铁 路 局 的 内 部 [MASK] 待 所 ， 位 于 火 车 站 出 站 口 处 ， [SEP]
解码:  招
序: 1 输入内容: [CLS] 这 个 争 议 也 已 经 若 干 年 了 ， 到 底 [MASK] 朝 亡 于 哪 一 年 ， 就 和 清 朝 应 该 [SEP]
解码:  明
序: 2 输入内容: [CLS] 很 好 的 酒 店 ， 紧 靠 海 边 ， 除 了 入 [MASK] 当 晚 网 络 不 好 ， 其 他 都 很 满 意 [SEP]
解码:  住
序: 3 输入内容: [CLS] 酒 店 房 间 还 算 可 以 ， 对 得 起 五 星 [MASK] 的 称 号 。 早 餐 品 种 也 算 是 丰 富 [SEP]
解码:  级
序: 4 输入内容: [CLS] 无 预 装 系 统 ， 且 主 板 [UNK] 没 有 打 入 [MASK] 证 书 ， 因 此 不 能 激 活 [UNK] 版 [UNK] ， [SEP]
解码:  [ U N K ]
序: 5 输入内容: [CLS] 2007 年 9 月 11 日 256 元 住 普 通 标 间 ， [MASK] 街 （ 其 它 房 型 已 无 ） 。 我 是 喜 [SEP]
解码:  临
序: 6 输入内容: [CLS] 2007 年 10 月 19 日 入 住 ， 总 体 感 觉 不 [MASK] ， 位 置 闹 中 取 静 ， 走 出 酒 店 斜 [SEP]
解码:  错
序: 7 输入内容: [CLS] 酒 店 生 意 清 淡 ， 大 堂 里 都 没 几 个 [MASK] 人 。 房 间 倒 是 不 小 ， 但 觉 得 被 [SEP]
解码:  客
序: 8 输入内容: [CLS] 感 动. 还 有 锥 心 的 悲 哀. 读 过 就 [MASK] 也 无 法 忘 怀. 很 怕 被 拍 成 影 视 [SEP]
解码:  再
序: 9 输入内容: [CLS] 不 错 ， 作 者 的 观 点 很 颠 覆 目 前 中 [MASK] 父 母 的 教 育 方 式 ， 其 实 古 人 们 [SEP]
解码:  国
序: 10 输入内容: [CLS] 从 04 年 到 06 年 ， 常 带 同 事 和 家 人 [MASK] 在 主 楼 1